In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tqdm import tqdm
import itertools
import datetime
from datetime import timedelta

In [2]:
with open(os.path.join('/home/tohone06/','result','paper_drop_srgc_mlpt_202310/train_replaced_with_syn.pkl'), 'rb') as f:
    test = pickle.load(f)

real_sta = test['raw_set'][0]
real_dyn = test['raw_set'][1] 

real_sta_test = test['test_set'][0]
real_dyn_test = test['test_set'][1] 


syn_sta = test['train_set'][0]
syn_dyn = test['train_set'][1]

In [3]:
lst = []
for k,g in enumerate(real_dyn):
    g['PTNO'] = k
    lst.append(g)
real_dyn = pd.concat(lst)

In [4]:
real_sta = real_sta.reset_index(drop = True)
real_sta = real_sta.reset_index()
real_sta = real_sta.rename(columns = {'index':'PTNO'})

In [5]:
real = pd.merge(real_sta,real_dyn, how = 'left', on= 'PTNO')
real = real.drop(columns = ['PTNO','LINE'])

In [6]:
real.columns

Index(['BSPT_SEX_CD', 'BSPT_BRYM', 'BSPT_FRST_DIAG_YMD', 'BSPT_FRST_DIAG_CD',
       'BSPT_DEAD_YMD', 'BSPT_STAG_VL', 'BSPT_T_STAG_VL', 'BSPT_N_STAG_VL',
       'BSPT_M_STAG_VL', 'BSPT_DEAD', 'seq_len', 'CEXM_RSLT_CONT', 'TIMESTAMP',
       'BPTH_BPSY_RSLT_CONT', 'IMPT_HM1E_RSLT_CD', 'IMPT_HS2E_RSLT_CD',
       'IMPT_HS6E_RSLT_CD', 'IMPT_HP2E_RSLT_CD', 'MLPT_KRES_RSLT_CD',
       'MLPT_NREX_RSLT_CD', 'MLPT_BRME_RSLT_CD', 'SGPT_PATL_STAG_VL',
       'SGPT_PATL_T_STAG_VL', 'SGPT_PATL_N_STAG_VL', 'SGPT_PATL_M_STAG_VL',
       'SGPT_SRMV_LN_CNT', 'SGPT_MTST_LN_CNT', 'SGPT_TUMR_BUDD_CD',
       'OPRT_CLCN_OPRT_KIND_CD', 'OPRT_CURA_RSCT_CD', 'CSTR_PRPS_CD',
       'CSTR_REGN_CD', 'CSTR_END_YMD', 'CSTR_NT', 'RDT_END_YMD'],
      dtype='object')

In [7]:
lst = []
for k,g in enumerate(real_dyn_test):
    g['PTNO'] = k
    lst.append(g)
real_dyn_test = pd.concat(lst)

real_sta_test = real_sta_test.reset_index(drop = True)
real_sta_test = real_sta_test.reset_index()
real_sta_test = real_sta_test.rename(columns = {'index':'PTNO'})

real_test = pd.merge(real_sta_test,real_dyn_test, how = 'left', on= 'PTNO')

real_test = real_test.drop(columns = ['PTNO','LINE'])

In [8]:
lst = []
for k,g in enumerate(syn_dyn):
    g['PTNO'] = k
    lst.append(g)
syn_dyn = pd.concat(lst)

In [9]:
syn_sta = syn_sta.reset_index(drop = True)
syn_sta = syn_sta.reset_index()
syn_sta = syn_sta.rename(columns = {'index':'PTNO'})

In [10]:
syn = pd.merge(syn_sta,syn_dyn, how = 'left', on= 'PTNO')

In [11]:
syn = syn.drop(columns = ['PTNO','LINE'])

In [12]:
numerical_cols =['BSPT_BRYM',
 'BSPT_FRST_DIAG_YMD',
 'BSPT_DEAD_YMD',
 'seq_len',
 'TIMESTAMP',
'CEXM_RSLT_CONT',
 'SGPT_SRMV_LN_CNT',
 'SGPT_MTST_LN_CNT',
 'CSTR_END_YMD',
 'CSTR_NT',
 'RDT_END_YMD']

categorical_cols = [c for c in real.columns if c not in numerical_cols]

In [13]:
#real
# 범주형 변수 One-Hot Encoding 및 NaN 값 처리
one_hot_encoded = pd.get_dummies(real[categorical_cols], dummy_na=True)


# 기존 데이터프레임과 One-Hot Encoding된 데이터 합치기
real_encoded = pd.concat([real, one_hot_encoded], axis=1)

# 기존 범주형 변수 열 삭제
real = real_encoded.drop(categorical_cols, axis=1)

# real_test
# 범주형 변수 One-Hot Encoding 및 NaN 값 처리
one_hot_encoded = pd.get_dummies(real_test[categorical_cols], dummy_na=True)

# 기존 데이터프레임과 One-Hot Encoding된 데이터 합치기
real_test_encoded = pd.concat([real_test, one_hot_encoded], axis=1)

# 기존 범주형 변수 열 삭제
real_test = real_test_encoded.drop(categorical_cols, axis=1)

#syn
# 범주형 변수 One-Hot Encoding 및 NaN 값 처리
one_hot_encoded = pd.get_dummies(syn[categorical_cols], dummy_na=True)

# 기존 데이터프레임과 One-Hot Encoding된 데이터 합치기
syn_encoded = pd.concat([syn, one_hot_encoded], axis=1)

# 기존 범주형 변수 열 삭제
syn = syn_encoded.drop(categorical_cols, axis=1)

In [14]:
real = real.fillna(0)
real_test = real_test.fillna(0)
syn = syn.fillna(0)

In [17]:
syn.rename(columns = {'BSPT_M_STAG_VL_0.0':'BSPT_M_STAG_VL_0','BSPT_M_STAG_VL_1.0':'BSPT_M_STAG_VL_1',
                      'BSPT_STAG_VL_0.0':'BSPT_STAG_VL_0','BSPT_STAG_VL_1.0':'BSPT_STAG_VL_1',
                      'BSPT_STAG_VL_2.0':'BSPT_STAG_VL_2','BSPT_STAG_VL_3.0':'BSPT_STAG_VL_3',
                     'BSPT_STAG_VL_4.0':'BSPT_STAG_VL_4','CEXM_RSLT_CONT_1.0':'CEXM_RSLT_CONT_1',
                     'CEXM_RSLT_CONT_2.0':'CEXM_RSLT_CONT_2','CEXM_RSLT_CONT_3.0':'CEXM_RSLT_CONT_3',
                     'SGPT_PATL_M_STAG_VL_0.0':'SGPT_PATL_M_STAG_VL_0','SGPT_PATL_M_STAG_VL_1.0':'SGPT_PATL_M_STAG_VL_1',
                     'SGPT_PATL_STAG_VL_0.0':'SGPT_PATL_STAG_VL_0','SGPT_PATL_STAG_VL_1.0':'SGPT_PATL_STAG_VL_1',
                     'SGPT_PATL_STAG_VL_2.0':'SGPT_PATL_STAG_VL_2','SGPT_PATL_STAG_VL_3.0':'SGPT_PATL_STAG_VL_3',
                     'SGPT_PATL_STAG_VL_4.0':'SGPT_PATL_STAG_VL_4'},inplace = True)

In [18]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 수치형 변수만 사용하여 NearestNeighbors 모델을 초기화합니다.

# NearestNeighbors 모델을 훈련합니다.
nn_model_real = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(real)

# 합성 데이터와 실제 데이터 간의 거리를 계산합니다.
distances_real_test, _ = nn_model_real.kneighbors(real_test)
distances_syn, _ = nn_model_real.kneighbors(syn)

# 최종 DCR 값을 계산합니다.
dcr_real_test_min = np.min(distances_real_test)
dcr_syn_min = np.mean(distances_syn)




/home/tohone06/.conda/envs/rtsgan/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/home/tohone06/.conda/envs/rtsgan/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [19]:
dcr_real_test_min = np.min(distances_real_test)
dcr_syn_min = np.min(distances_syn)

In [20]:
np.round(dcr_real_test_min,2)

2.45

In [21]:
np.round(dcr_syn_min,2)

3.46